In [1]:
% matplotlib inline
% load_ext autoreload
% autoreload 2

In [2]:
from __future__ import print_function
import tensorflow as tf
import numpy as np
from deep_visual_odometry.kitti_utils import KITTIdata
from matplotlib import pyplot as plt

In [3]:
basedir = 'dataset'

In [4]:
from deep_visual_odometry.odometry_model import OdomModel

In [5]:
batch_size = 256         # Sequences per batch
num_steps = 8         # Number of sequence steps per batch
rnn_size = 64          # Size of hidden layers in rnn_cell
num_layers = 2           # Number of hidden layers
learning_rate = 0.001

In [6]:
kitti_data = KITTIdata(basedir, 
                       sequences = ['01'],
                       sequence_len = num_steps,
                       val_frac = 0,
                       test_frac = 0,
                       img_size=(150,50))

completed load sequence 01 data


In [7]:
mean_velo = np.mean(kitti_data.velocities['01'],axis = 0)
std_velo = np.std(kitti_data.velocities['01'],axis = 0)

mean_poses = np.mean(kitti_data.poses['01'],axis = 0)
std_poses = np.std(kitti_data.poses['01'],axis = 0)

In [8]:
kitti_data.velocities['01'] = kitti_data.velocities['01']-mean_velo
kitti_data.velocities['01'] /= std_velo

In [9]:
kitti_data.poses['01'] = kitti_data.poses['01']-mean_poses
kitti_data.poses['01'] /= std_poses

In [10]:
model = OdomModel(batch_size = 256, 
                  num_steps = 8,
                  cell_type = 'LSTM',
                  rnn_size = 128,
                  num_layers = 2,
                  learning_rate = 0.001)
model.train(kitti_data, max_count = 10000, save_every_n = 1000)

output size (256, 8, 128)
seq_output shape (256, 8, 128)
x (2048, 128)
mse_w (128, 3)
shape of logits (256, 8, 3)
shape of targets (256, 8, 3)
step: 500  loss: 0.4560  0.0876 sec/batch
step: 1000  loss: 0.3197  0.0876 sec/batch
step: 1500  loss: 0.3253  0.0888 sec/batch
step: 2000  loss: 0.2897  0.0865 sec/batch
step: 2500  loss: 0.2956  0.0812 sec/batch
step: 3000  loss: 0.2791  0.0885 sec/batch
step: 3500  loss: 0.2537  0.0851 sec/batch
step: 4000  loss: 0.2596  0.0855 sec/batch
step: 4500  loss: 0.2466  0.0826 sec/batch
step: 5000  loss: 0.2634  0.0872 sec/batch
step: 5500  loss: 0.2376  0.0874 sec/batch
step: 6000  loss: 0.2289  0.0811 sec/batch
step: 6500  loss: 0.2226  0.0786 sec/batch
step: 7000  loss: 0.2396  0.0844 sec/batch
step: 7500  loss: 0.2356  0.0787 sec/batch
step: 8000  loss: 0.2006  0.0832 sec/batch
step: 8500  loss: 0.2379  0.0807 sec/batch
step: 9000  loss: 0.2209  0.0809 sec/batch
step: 9500  loss: 0.2116  0.0867 sec/batch
step: 10000  loss: 0.2032  0.0798 sec/bat

In [11]:
kitti_data.dataset_len['01']

1101

In [12]:
inputs_train, velocities_train, poses_train = kitti_data.load_data_train()

609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
855
856
857
858


In [13]:
inputs_train.shape

(485, 8, 50, 150, 6)